In [ ]:
import os
import subprocess

home = os.path.expanduser('~')
os.environ['ROS_DOMAIN_ID']="15"
#os.environ['RMW_IMPLEMENTATION'] = 'rmw_cyclonedds_cpp'
os.environ['RMW_IMPLEMENTATION'] = 'rmw_fastrtps_cpp'
print(os.environ['ROS_DOMAIN_ID'])
print(os.environ['RMW_IMPLEMENTATION'])
print(os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'])

In [ ]:
list=subprocess.run(['ros2','topic','list'],capture_output=True)

In [ ]:
list.stdout.decode().split('\n')[0]

In [ ]:
! printenv | grep 'ROS'

In [ ]:
import time
for i in range(17):
    os.environ['ROS_DOMAIN_ID']=str(i)
    print(os.environ['ROS_DOMAIN_ID'])
    time.sleep(0.1)
    list = subprocess.run(['ros2','topic','list'],capture_output=True)
    print(list.stdout.decode().split('\n')[0])

If that did not work - lets check the RMW implementation - it should be fastrtps

In [ ]:
print(os.environ['RMW_IMPLEMENTATION'])
os.environ['RMW_IMPLEMENTATION']= 'rmw_fastrtps_cpp'
print(os.environ['RMW_IMPLEMENTATION'])

If that is fine, then lets check the multicast

In [ ]:
print(os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'])

It should read the same as the file below if there is no response, then run this line - and it will set it to a file in your Documents folder

In [ ]:
os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'] = home +'/Documents/disable_multicast.xml'

Next you need to save the list of IP addresses - MAKE SURE to change the file name to the path above

In [ ]:
%%writefile /home/parallels/Documents/disable_multicast.xml

<?xml version="1.0" encoding="UTF-8" ?>
<profiles xmlns="http://www.eprosima.com/XMLSchemas/fastRTPS_Profiles">
   <participant profile_name="unicast_connection" is_default_profile="true">
       <rtps>
           <builtin>
               <metatrafficUnicastLocatorList>
                   <locator/>
               </metatrafficUnicastLocatorList>
               <initialPeersList>
                    <locator>
                       <udpv4>
                           <address>127.0.0.1</address>
                           <!--Localhost -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.156.115 </address>
                           <!--Syndrome -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.152.174</address>
                           <!--Dory -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.157.95</address>
                           <!--Nemo -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.150.128</address>
                           <!--Remy -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.146.11</address>
                           <!--EdnaMode  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.157.182</address>
                           <!--Woody  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.156.217</address>
                           <!--Buzz -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.154.102</address>
                           <!--MikeWazowski -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.147.32</address>
                           <!--LightningMcQueen  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.144.147</address>
                           <!--Mater -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.153.15</address>
                           <!--Dash -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.150.175</address>
                           <!--ElastaGirl -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.92.65</address>
                           <!--Pig -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.153.159</address>
                           <!--PotatoHead -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.148.45</address>
                           <!--Dot -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.154.21</address>
                           <!--Sully -->
                       </udpv4>
                   </locator>
               </initialPeersList>
           </builtin>
       </rtps>
   </participant>
</profiles>
<!--ros2 daemon stop 🤔️ source ~/.bashrc 🤔️ ros2 topic list-->

Stop the daemon and start it up again

In [ ]:
!ros2 daemon stop

Then restart everything by asking for a topic list - sometimes this takes up to 1-2 min to update

In [ ]:
!export ROS_DOMAIN_ID=4

In [ ]:
!ros2 topic list

For even more fun - check all the battery levels

In [ ]:
import os
import time

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import BatteryState

class battery(Node):
    def __init__(self, name):
        super().__init__('battery_subscriber')
        self.subscription = self.create_subscription(BatteryState, 
                    '/' + name + '/battery_state', self.callback, qos_profile_sensor_data)
    
    def callback(self, msg: BatteryState):
        charge = 100*msg.percentage
        print(' %0.1f ' % charge, end='')


names = ['rogers','Syndrome','Dory','Nemo','Remy','EdnaMode','Woody','Buzz','MikeWazowski','LightningMcQueen','Mater','Dash','ElastGirl','Pig','PotatoHead','Dot','Sully']


rclpy.init()

for i in range(4):
    bs = battery(names[i])
    print('checking %s (%d)' % (names[i],i), end='')
    os.environ['ROS_DOMAIN_ID'] = str(i)
    try:
        rclpy.spin_once(bs, timeout_sec = 10)
    except KeyboardInterrupt:
        print('\nCaught keyboard interrupt')
        break
    finally:
        print(" Done")
        bs.destroy_node()
    
rclpy.shutdown()